### Importing Library

In [68]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

### Reading the dataset and performing necesary operations

In [69]:
df=pd.read_csv('ratings.csv')
df['TotalTime']=100
df['WatchTime'] = df.apply(lambda x: random.randint(0, 100))
df['WatchTime'] = df['WatchTime'].apply(lambda x: random.randint(0, 100) if pd.isnull(x) else x)
df.columns=['User', 'MovieID', 'Rating', 'timestamp', 'TotalTime', 'WatchTime']

# Calculate the adjusted rating based on watch time
df['AdjustedRating'] = df.apply(lambda row: row['Rating'] if row['WatchTime'] == row['TotalTime']
                                else (row['WatchTime'] / row['TotalTime']) * row['Rating'], axis=1)

### Train and Test split

In [70]:
# Pivot table to reshape the data
pivot_table = df.pivot(index='User', columns='MovieID', values='AdjustedRating')

# Split the data into training and test sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Calculate the item-item similarity matrix based on adjusted ratings
combined_similarity = cosine_similarity(pivot_table.fillna(0))

### Function creation 

In [71]:
# Function to get similar movies for a given movie
def get_similar_movies(movie_id):
    similar_movies = []
    movie_index = pivot_table.index.get_loc(movie_id)
    for idx, similarity in enumerate(combined_similarity[movie_index]):
        if similarity > 0 and similarity < 1:
            similar_movies.append(pivot_table.columns[idx])
    return similar_movies

### Example usage: Get similar movies for movie

In [72]:
similar_movies = get_similar_movies(1)
print("Similar movies for 'Movie1':", similar_movies)

Similar movies for 'Movie1': [1, 4, 5, 7, 9, 15, 17, 19, 20, 21, 22, 23, 25, 26, 27, 30, 31, 32, 34, 35, 36, 37, 39, 40, 42, 44, 49, 50, 57, 59, 60, 62, 64, 70, 72, 74, 78, 80, 81, 82, 83, 84, 86, 88, 94, 95, 97, 101, 102, 103, 105, 107, 111, 112, 114, 121, 122, 131, 132, 135, 145, 148, 150, 153, 154, 157, 161, 168, 170, 172, 174, 175, 177, 181, 186, 188, 198, 200, 201, 208, 209, 213, 215, 218, 220, 222, 223, 224, 225, 229, 234, 238, 239, 240, 241, 244, 247, 249, 251, 253, 255, 256, 259, 260, 263, 269, 270, 274, 275, 279, 280, 281, 285, 291, 294, 299, 307, 313, 314, 315, 316, 318, 320, 321, 324, 325, 326, 328, 329, 331, 333, 334, 337, 338, 340, 342, 343, 344, 345, 346, 347, 349, 352, 354, 360, 361, 363, 365, 370, 372, 373, 376, 377, 379, 381, 387, 388, 389, 390, 392, 405, 408, 409, 410, 412, 419, 421, 422, 429, 431, 434, 436, 437, 438, 440, 441, 444, 445, 448, 451, 453, 454, 455, 457, 458, 463, 470, 471, 474, 475, 480, 482, 483, 486, 488, 491, 493, 494, 505, 510, 511, 513, 514, 516, 51

### Evaluate the model using mean absolute error (MAE)

In [73]:
predictions = []
for _, row in test_data.iterrows():
    user = row['User']
    movie = row['MovieID']
    predicted_rating = pivot_table.loc[user, movie]
    predictions.append(predicted_rating)

actual_ratings = test_data['AdjustedRating']
mae = mean_absolute_error(actual_ratings, predictions)
print("Mean Absolute Error (MAE):", mae)


Mean Absolute Error (MAE): 0.0
